This code is basically the same as model_training.py except written in a jupyter notebook so it can work with google colab.

In [ ]:
!pip install gdown #<-- lets you get drive files into colab using their file ids ig
!pip install tensorflow-gpu #<-- leverages tensor flows GPU library which make things faster

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [ ]:
!gdown --id #insert id here# --output ##insert file name here
!gdown --id #insert id here# --output ##insert file name here

In [ ]:
#Change file names in the following two
cook_df = pd.read_csv('./samples/cook.csv')
flick_df = pd.read_csv('./samples/flickup.csv')

#getting rid of last 5 elements from cook_df
flick_df = flick_df[0:len(flick_df)-10]

#since one second is about 118 rows, im getting rid of the extra rows that would make it evenly divisible
cook_df = cook_df[0:len(cook_df)-len(cook_df)%118]
flick_df = flick_df[0:len(flick_df)-len(flick_df)%118]

print(cook_df.info())
print(flick_df.info())

In [ ]:
cook_df['Time'] = cook_df['Time'].astype(float)
flick_df['Time'] = flick_df['Time'].astype(float)


cook_df = cook_df.drop(columns = ['Gx', 'Gy', 'Gz'])
flick_df = flick_df.drop(columns = ['Gx', 'Gy', 'Gz'])

cook_df['Class'] = 'cook'
flick_df['Class'] = 'flick'

cook_df.info()
flick_df.info()

In [ ]:
train_cook_df = cook_df[0:int(len(cook_df)*.9)]
test_cook_df = cook_df[int(len(cook_df)*.9):]

train_flick_df = flick_df[0:int(len(flick_df)*.9)]
test_flick_df = flick_df[int(len(flick_df)*.9):]

window = 49

In [ ]:
def formatData(X,Y, window):
    Xarr = []
    Yarr = []

    for i in range(0, len(X) - window, int(window/2)):
        Xarr.append(X.iloc[i:i+window].values)  
        Yarr.append(Y.iloc[0])

    return np.array(Xarr), np.array(Yarr)

In [ ]:
#test data 
X_train1, Y_train1 = formatData(train_cook_df[['Ax', 'Ay', 'Az']], train_cook_df[['Class']], window)
X_train2, Y_train2 = formatData(train_flick_df[['Ax', 'Ay', 'Az']], train_flick_df[['Class']], window)

#train data
X_test1, Y_test1 = formatData(test_cook_df[['Ax', 'Ay', 'Az']], test_cook_df[['Class']], window)
X_test2, Y_test2 = formatData(test_flick_df[['Ax', 'Ay', 'Az']], test_flick_df[['Class']], window)


#manually encoding because i didnt do it earlier and it would be a pain to go back. 
#will change when i add more classes
for i in range(len(Y_train1)):
    Y_train1[i] = 0
    Y_train2[i] = 1

for i in range(len(Y_test1)):
    Y_test1[i] = 0
    Y_test2[i] = 1

X_train_final = np.concatenate((X_train1, X_train2))
Y_train_final = np.concatenate((Y_train1, Y_train2))

X_test_final = np.concatenate((X_test1, X_test2))
Y_test_final = np.concatenate((Y_test1, Y_test2))


print(np.shape(X_train_final))

In [ ]:
model = keras.Sequential()

#add bidirection lstm layer
model.add(
    keras.layers.Bidirectional(
        keras.layers.LSTM(
            units=128,
            input_shape = [X_train_final.shape[1], X_train_final.shape[2]]
        )
    )
)

#adding a dropout layer (prevents overfitting - look it up)
model.add(keras.layer.Dropout(rate=0.5))

#dense layer is the layer of nodes
model.add(keras.layers.Dense(units=128, activation='relu'))

#this layer will return the output prediction. Softmax puts "confidence" between 0 and 1
model.add(keras.layers.Dense(Y_train_final.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentrropy', optimizer='adam', metrics=['acc'])



In [ ]:
#batch size will be played around with bc my data is small :/
trained_model = model.fit(
    X_train_final, Y_train_final,
    epochs =20,
    batch_size=16,
    validation_split= 0.1,
    shuffle=True
)